<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=121ce8b45c7346106c12a01ea8fb10a2918ed278df5152459fbc4d28a331156a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:


      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-12 12:17:04
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16


-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: 1.24 L (0.86%)
Current PnL: -21.59 L (-14.15%)
CY Booked + Current PnL: -8.07 L (-5.29%)
-------------------
Total profit:  1.89 L
Total loss:  -23.48 L
-------------------
Total Booked + Current PnL: 18.91 L (15.05%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.32%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 88.26 L (60.85%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,101.71,51.0,M-SC,2.55,86899.0,-13878.0,13973.0,0.26,-13.77,16.08,0.09,245.0,-0.99,0.61,14.17,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-11.06,43.0,X-MC,3.32,87810.0,-2868.0,17773.0,-0.95,-3.16,20.24,16.44,101.0,-0.16,0.61,16.37,X40,ATH,PHARMA
42,ITC,452.00,-38.34,46.0,X-LC,0.82,199047.0,-1091.0,21975.0,0.05,-0.55,11.04,10.44,4.0,-0.05,1.39,4.99,X40,NTT,FMCG
34,ICICIGI,2252.93,-13.14,57.0,X-MC,7.55,187525.0,13205.0,21997.0,1.02,7.58,11.73,20.19,91.0,0.60,1.31,23.83,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-2.62,65.0,X-LC,8.58,299637.0,34211.0,22593.0,0.98,12.89,7.54,21.40,11.0,1.51,2.09,20.39,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,ROUTE,2227.21,-56.81,41.0,H-SC,23.19,71296.0,-61770.0,153650.0,5.04,-46.42,215.51,69.05,118.0,-0.40,0.50,5.04,SR,ATH,IT
39,INDUSINDBK,1800.00,297.06,77.0,L-MC,7.12,48735.0,-28472.0,53867.0,3.50,-36.88,110.53,32.89,258.0,-0.53,0.34,34.21,XR,NTT,BANKS
16,CAMS,4762.00,-2.28,58.0,X-SC,1.23,261888.0,10032.0,52404.0,3.38,3.98,20.01,24.79,122.0,0.19,1.82,28.04,X40N,NTT,MISC
72,TATAELXSI,9161.00,-23.37,53.0,H-MC,6.32,103569.0,-28266.0,70489.0,3.17,-21.44,68.06,32.03,98.0,-0.40,0.72,16.01,OX40N,NTT,IT
49,LTIM,7201.88,6.94,70.0,H-LC,10.17,265095.0,18944.0,58984.0,3.16,7.70,22.25,31.66,16.0,0.32,1.85,47.09,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,671.64,-18.39,47.0,X-LC,5.68,294054.0,-21788.0,134441.0,-2.06,-6.90,45.72,35.67,5.0,-0.16,2.05,6.00,X40N,ATH,FMCG
56,RAJOOENG,143.10,-45.47,36.0,H-SC,17.50,82500.0,-20000.0,60596.0,-1.42,-19.51,73.45,39.61,114.0,-0.33,0.57,0.76,AR,ATH,MISC
2,ABBOTINDIA,35195.00,-11.06,43.0,X-MC,3.32,87810.0,-2868.0,17773.0,-0.95,-3.16,20.24,16.44,101.0,-0.16,0.61,16.37,X40,ATH,PHARMA
8,ATULAUTO,844.00,3564.86,51.0,M-SC,4.00,164543.0,-32589.0,122420.0,-0.77,-16.53,74.40,45.57,236.0,-0.27,1.15,16.66,XY24,NTT,AUTO
75,TMPV,1065.00,NaN,52.0,NaN,NaN,211123.0,-186521.0,344806.0,-0.77,-46.91,163.32,39.81,54.0,-0.54,1.47,3.48,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.72,152253.0,1308.0,108145.0,1.61,0.87,71.03,72.51,53.0,0.01,1.06,6.94,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-8.62,41.0,H-MC,6.00,210911.0,1337.0,73334.0,1.65,0.64,34.77,35.63,NaN,0.02,1.47,17.42,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,101.71,51.0,M-SC,2.55,86899.0,-13878.0,13973.0,0.26,-13.77,16.08,0.09,245.0,-0.99,0.61,14.17,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,129.35,79.0,M-SC,4.45,150369.0,-24190.0,24224.0,0.84,-13.86,16.11,0.02,221.0,-1.00,1.05,30.79,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-67.97,44.0,H-SC,2.33,220968.0,-48699.0,85028.0,0.68,-18.06,38.48,13.47,138.0,-0.57,1.54,12.36,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.37,53.0,H-MC,6.32,103569.0,-28266.0,70489.0,3.17,-21.44,68.06,32.03,98.0,-0.40,0.72,16.01,OX40N,NTT,IT
66,SIS,528.0,2022.28,47.0,H-SC,2.22,85654.0,-25378.0,48986.0,0.57,-22.86,57.19,21.26,156.0,-0.52,0.60,15.73,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,ZYDUSLIFE,1286.17,-8.62,41.0,H-MC,6.00,210911.0,1337.0,73334.0,1.65,0.64,34.77,35.63,NaN,0.02,1.47,17.42,AR,ATH,PHARMA
84,WIPRO,420.00,-13.34,55.0,M-LC,5.72,152253.0,1308.0,108145.0,1.61,0.87,71.03,72.51,53.0,0.01,1.06,6.94,XR,NTT,IT
37,INDIAMART,4810.62,-52.59,61.0,H-SC,4.90,132463.0,9127.0,122502.0,1.93,7.40,92.48,106.72,119.0,0.07,0.92,32.51,AR,ATH,MISC
61,SAMMAANCAP,326.00,-192.05,54.0,M-SC,35.62,159660.0,10440.0,133747.0,0.59,7.00,83.77,96.62,208.0,0.08,1.11,72.79,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.59,61.0,H-SC,4.90,132463.0,9127.0,122502.0,1.93,7.40,92.48,106.72,119.0,0.07,0.92,32.51,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-8.62,41.0,H-MC,6.00,210911.0,1337.0,73334.0,1.65,0.64,34.77,35.63,NaN,0.02,1.47,17.42,AR,ATH,PHARMA
84,WIPRO,420.00,-13.34,55.0,M-LC,5.72,152253.0,1308.0,108145.0,1.61,0.87,71.03,72.51,53.0,0.01,1.06,6.94,XR,NTT,IT
83,WHIRLPOOL,2270.00,-45.46,36.0,M-SC,8.83,113050.0,-1748.0,95787.0,-0.73,-1.52,84.73,81.92,223.0,-0.02,0.79,33.83,XR,NTT,DURABLES
47,KPIGREEN,731.05,2.88,45.0,H-SC,10.18,122800.0,-2497.0,61425.0,0.21,-1.99,50.02,47.03,141.0,-0.04,0.86,53.76,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86918.0,-41752.0,87048.0,0.03,-32.45,100.15,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
26,GLAXO,3466.20,-20.84,29.0,X-MC,4.86,200160.0,876.0,77142.0,1.04,0.44,38.54,39.15,96.0,0.01,1.39,29.57,X40,ATH,PHARMA
25,GILLETTE,11206.78,-16.62,31.0,X-MC,5.70,256275.0,2379.0,79932.0,3.11,0.94,31.19,32.42,105.0,0.03,1.79,16.18,X40,ATH,FMCG
1,ABB,7934.00,-43.17,31.0,H-LC,5.04,237432.0,-24187.0,143409.0,-0.64,-9.25,60.40,45.57,7.0,-0.17,1.65,1.03,AR,NTT,ELECTRICAL
45,JSWINFRA,342.00,-23.96,32.0,X-MC,4.70,191025.0,-9322.0,39829.0,-0.14,-4.65,20.85,15.23,178.0,-0.23,1.33,26.10,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.34,46.0,X-LC,0.82,199047.0,-1091.0,21975.0,0.05,-0.55,11.04,10.44,4.0,-0.05,1.39,4.99,X40,NTT,FMCG
53,PGHH,17907.65,-32.66,33.0,X-MC,1.12,194880.0,-5940.0,73743.0,0.57,-2.96,37.84,33.76,80.0,-0.08,1.36,2.09,X40,ATH,FMCG
65,SIEMENS,4671.50,-6.04,37.0,H-LC,1.69,151115.0,-34980.0,82463.0,0.01,-18.80,54.57,25.51,15.0,-0.42,1.05,11.36,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-8.45,41.0,X-MC,3.29,305739.0,-22252.0,128777.0,0.85,-6.78,42.12,32.48,92.0,-0.17,2.13,3.87,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-5.81,63.0,X-LC,3.44,246820.0,12676.0,35937.0,1.45,5.41,14.56,20.77,86.0,0.35,1.72,12.71,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86918.0,-41752.0,87048.0,0.03,-32.45,100.15,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,33.0,X-SC,37.13,48217.0,-16789.0,167718.0,0.77,-25.83,347.84,232.18,198.0,-0.10,0.34,0.77,XY24,NTT,MISC
1,ABB,7934.00,-43.17,31.0,H-LC,5.04,237432.0,-24187.0,143409.0,-0.64,-9.25,60.40,45.57,7.0,-0.17,1.65,1.03,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.66,33.0,X-MC,1.12,194880.0,-5940.0,73743.0,0.57,-2.96,37.84,33.76,80.0,-0.08,1.36,2.09,X40,ATH,FMCG
10,BAJAJHFL,181.50,-15.96,39.0,X-MC,6.32,177235.0,-23271.0,120431.0,0.36,-11.61,67.95,48.45,90.0,-0.19,1.23,2.20,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,33.0,X-SC,37.13,48217.0,-16789.0,167718.0,0.77,-25.83,347.84,232.18,198.0,-0.10,0.34,0.77,XY24,NTT,MISC
58,RELAXO,1176.00,-40.07,45.0,X-SC,3.69,81658.0,-63502.0,139431.0,0.57,-43.75,170.75,52.31,136.0,-0.46,0.57,9.30,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.06,43.0,X-MC,3.32,87810.0,-2868.0,17773.0,-0.95,-3.16,20.24,16.44,101.0,-0.16,0.61,16.37,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86918.0,-41752.0,87048.0,0.03,-32.45,100.15,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
33,HONAUT,58357.33,-24.36,36.0,X-SC,0.93,106620.0,-21318.0,68450.0,0.10,-16.66,64.20,36.84,143.0,-0.31,0.74,9.15,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.44,62.0,X-LC,8.96,292904.0,-53052.0,119739.0,2.26,-15.33,40.88,19.28,1.0,-0.44,2.04,8.28,X40,ATH,IT
40,INFY,2275.00,-15.29,67.0,X-LC,4.54,330448.0,17560.0,154121.0,1.38,5.61,46.64,54.87,3.0,0.11,2.30,14.36,X40,BTT,IT
42,ITC,452.00,-38.34,46.0,X-LC,0.82,199047.0,-1091.0,21975.0,0.05,-0.55,11.04,10.44,4.0,-0.05,1.39,4.99,X40,NTT,FMCG
81,VBL,671.64,-18.39,47.0,X-LC,5.68,294054.0,-21788.0,134441.0,-2.06,-6.90,45.72,35.67,5.0,-0.16,2.05,6.00,X40N,ATH,FMCG
1,ABB,7934.00,-43.17,31.0,H-LC,5.04,237432.0,-24187.0,143409.0,-0.64,-9.25,60.40,45.57,7.0,-0.17,1.65,1.03,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,41.0,L-SC,26.23,76320.0,-37229.0,77282.0,-0.38,-32.79,101.26,35.27,268.0,-0.48,0.53,86.80,XR,NTT,HOTELS
7,ASIANTILES,137.00,7572.22,68.0,L-SC,15.00,84169.0,-9641.0,86256.0,0.24,-10.28,102.48,81.67,269.0,-0.11,0.59,63.00,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.15,50.0,H-SC,7.88,91890.0,-6090.0,27696.0,1.27,-6.22,30.14,22.05,152.0,-0.22,0.64,33.57,XR,ATH,FINANCE
47,KPIGREEN,731.05,2.88,45.0,H-SC,10.18,122800.0,-2497.0,61425.0,0.21,-1.99,50.02,47.03,141.0,-0.04,0.86,53.76,MH,ATH,POWER
37,INDIAMART,4810.62,-52.59,61.0,H-SC,4.90,132463.0,9127.0,122502.0,1.93,7.40,92.48,106.72,119.0,0.07,0.92,32.51,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,521.00,71.35,60.0,H-SC,6.60,148360.0,-34415.0,145482.0,2.14,-18.83,98.06,60.77,125.0,-0.24,1.03,33.61,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7572.22,68.0,L-SC,15.00,84169.0,-9641.0,86256.0,0.24,-10.28,102.48,81.67,269.0,-0.11,0.59,63.00,XR,NTT,CERAMICS
38,INDIGOPNTS,1408.00,129.35,79.0,M-SC,4.45,150369.0,-24190.0,24224.0,0.84,-13.86,16.11,0.02,221.0,-1.00,1.05,30.79,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,23.98,70.0,X-SC,6.88,219145.0,28139.0,29563.0,1.18,14.73,13.49,30.21,157.0,0.95,1.53,38.69,X40N,NTT,FINANCE
6,ASIANPAINT,3465.66,-7.05,75.0,X-LC,6.83,234784.0,-16984.0,70200.0,0.41,-6.75,29.90,21.13,27.0,-0.24,1.64,26.29,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.91
1,25,44.59
2,50,76.29


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.57
LC    27.93
MC    27.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.53
X40      20.73
X40N     12.41
XY25     10.18
XR        9.71
AR        9.13
OX40N     6.59
X200      1.85
SR        1.03
MH        0.86
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.97
X-MC    22.89
X-LC    19.94
M-SC    12.54
X-SC     5.61
H-LC     4.55
H-MC     3.82
M-LC     2.45
L-SC     1.45
L-LC     0.99
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.89,-3.91,37.51
IT,12.80,-14.47,75.74
FINANCE,12.14,-8.57,57.78
MISC,7.69,-17.56,76.86
PAINTS,5.78,-11.40,28.01
ELECTRICAL,5.57,-12.83,52.83
INSURANCE,4.47,1.45,33.28
PHARMA,3.47,-0.13,33.73
AUTO,3.22,-50.48,112.77


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3214044.0,20
XR,1264185.0,13
AR,1259611.0,10
X40,994335.0,14
X40N,686759.0,9
OX40N,540084.0,9
XY25,473936.0,7
SR,272381.0,2
MH,61425.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3393397.0,24
X-MC,1398954.0,16
M-SC,1244523.0,15
X-LC,793128.0,11
X-SC,544614.0,6
H-MC,323914.0,3
H-LC,284856.0,3
L-SC,252776.0,3
M-LC,154095.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1167947.0      6
           AR         837635.0      5
           XR         795324.0      7
M-SC       XY24       723023.0      6
X-MC       XY24       564065.0      4
           X40        489687.0      8
X-LC       X40        417600.0      5
X-SC       X40N       289848.0      4
H-SC       SR         272381.0      2
           OX40N      258685.0      3
X-LC       X40N       236651.0      3
H-LC       AR         225872.0      2
X-MC       XY25       184942.0      2
H-MC       XY24       180091.0      1
X-SC       XY24       167718.0      1
L-SC       XR         163538.0      2
X-MC       X40N       160260.0      2
M-SC       XR         143311.0      2
           XY25       133747.0      1
           AR         122770.0      2
           OX40N      121672.0      4
M-LC       XR         108145.0      1
L-SC       OX40N       89238.0      1
X-SC       X40         87048.0      1
H-MC       AR          73334.0      1
X-LC       XY25        72483.0      2
H-MC       OX40N       70489.0      1
X-LC       XY24        66394.0      1
H-SC       MH          61425.0      1
H-LC       X200        58984.0      1
L-MC       XR          53867.0      1
M-LC       XY25        45950.0      1
L-LC       XY25        36814.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
